# 비즈덤 활성화하기

1. 아나콘다 프롬프트를 키고 activate 가상환경(pytorch) 를 실행한다.
2. python -m visdom.server 입력해서 서버를 킨다.
3. 크롬에서 http://localhost:8097/ 를 주소창에 킨다.
4. import visdom / vis = visdom.Visdom() 을 입력한다.

In [16]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

# import visdom

In [17]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

## define loss_tracker

loss 를 추적하는 함수를 정의해서, 시각화 하고싶다!


In [18]:
def loss_tracker(loss_plot, loss_value, num): 
    # loss_plot = 업데이트 할 
    # loss_value = loss 값
    # num = 몇번 진행되고 있는지
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [20]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [21]:
#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform = transforms.ToTensor(),
                        download=True)

In [22]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle =True,
                                          drop_last=True)

In [23]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out
            

In [24]:
model = CNN().to(device)

value = (torch.Tensor(1,1,28,28)).to(device)
print( (model(value)).shape ) 

torch.Size([1, 10])


In [25]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

## make plot

In [27]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), 
                    # 얘를 기반으로 어쩃든 업데이트가 된다. 일단 하나 넣어야하니까 0을 넣고
                    opts=dict(title='loss_tracker', # 이름을 뭐로 지을거고 
                              legend=['loss'], # legend 를 표시할지
                              showlegend=True)) 

## train with loss_tracker

In [ ]:
# 돌리게 되면 visdom 에서 upadte 가 된다.

In [15]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    # loss tracker 가 loss plt 을 update 할건데 
    # 우리는 위의 값들을 torch.Trnsor 형태로 받아야 하므로 감싸준 모습니다.
    # 한 에폭이 끝날떄마다 cost 를 평균을 넣겠다! 의 의미
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')

[Epoch:1] cost = 0.12043287605047226
[Epoch:2] cost = 0.041646845638751984


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())